<a href="https://colab.research.google.com/github/Pranv-Shrma/Fine-Tuning-Whisper-Small-for-Hindi-ASR/blob/main/Whisper_Finetune_Final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install uv

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.4/21.4 MB 96.5 MB/s eta 0:00:00


In [ ]:
!uv pip install --quiet transformers
!uv pip install --quiet librosa
!uv pip install --quiet torch==2.8.0 torchvision==0.23.0 torchaudio==2.8.0 --index-url https://download.pytorch.org/whl/cu128
!uv pip install --quiet datasets

Using Python 3.12.12 environment at: /usr
Audited 1 package in 332ms
Using Python 3.12.12 environment at: /usr
Audited 1 package in 95ms
Using Python 3.12.12 environment at: /usr
Audited 3 packages in 107ms
Using Python 3.12.12 environment at: /usr
Audited 1 package in 96ms


In [ ]:
# Ensuring the installation of Pytorch
!pip show torch

Name: torch
Version: 2.8.0+cu126
Summary: Tensors and Dynamic neural networks in Python with strong GPU acceleration
Home-page: https://pytorch.org/
Author: PyTorch Team
Author-email: packages@pytorch.org
License: BSD-3-Clause
Location: /usr/local/lib/python3.12/dist-packages
Requires: filelock, fsspec, jinja2, networkx, nvidia-cublas-cu12, nvidia-cuda-cupti-cu12, nvidia-cuda-nvrtc-cu12, nvidia-cuda-runtime-cu12, nvidia-cudnn-cu12, nvidia-cufft-cu12, nvidia-cufile-cu12, nvidia-curand-cu12, nvidia-cusolver-cu12, nvidia-cusparse-cu12, nvidia-cusparselt-cu12, nvidia-nccl-cu12, nvidia-nvjitlink-cu12, nvidia-nvtx-cu12, setuptools, sympy, triton, typing-extensions
Required-by: accelerate, fastai, peft, sentence-transformers, timm, torchaudio, torchdata, torchvision


In [ ]:
!uv pip install pandas

Using Python 3.12.12 environment at: /usr
Audited 1 package in 93ms


## Mounting Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Whisper_Finetune/FT Data - data.csv')

In [ ]:
df.head()

In [ ]:
# Renaming to Hugging Face convention
df = df.rename(columns={
    'rec_url_gcp': 'audio',
    'transcription_url_gcp': 'transcription'
})

In [ ]:
df2 = df[['audio','transcription']]
df2.head(5)

,audio,transcription
0,https://storage.googleapis.com/joshtalks-data-...,https://storage.googleapis.com/joshtalks-data-...
1,https://storage.googleapis.com/joshtalks-data-...,https://storage.googleapis.com/joshtalks-data-...
2,https://storage.googleapis.com/joshtalks-data-...,https://storage.googleapis.com/joshtalks-data-...
3,https://storage.googleapis.com/joshtalks-data-...,https://storage.googleapis.com/joshtalks-data-...
4,https://storage.googleapis.com/joshtalks-data-...,https://storage.googleapis.com/joshtalks-data-...


## Installing the audio files from url path to perform audio processing

In [ ]:
import os
import requests
import pandas as pd

def download_audio(row, audio_col="audio", download_dir="audio_files"):
    os.makedirs(download_dir, exist_ok=True)
    audio_url = row[audio_col]
    basename = os.path.basename(audio_url)
    local_path = os.path.join(download_dir, basename)
    if not os.path.exists(local_path):
        # Download only if not already present
        try:
            r = requests.get(audio_url)
            with open(local_path, 'wb') as f:
                f.write(r.content)
        except Exception as e:
            print(f"Failed to download {audio_url}: {e}")
    return local_path

df2["audio"] = df2.apply(download_audio, axis=1)

/tmp/ipython-input-1177154978.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2["audio"] = df2.apply(download_audio, axis=1)


In [ ]:
df2.head(5)

,audio,transcription
0,audio_files/825780_audio.wav,https://storage.googleapis.com/joshtalks-data-...
1,audio_files/825727_audio.wav,https://storage.googleapis.com/joshtalks-data-...
2,audio_files/988596_audio.wav,https://storage.googleapis.com/joshtalks-data-...
3,audio_files/990175_audio.wav,https://storage.googleapis.com/joshtalks-data-...
4,audio_files/526266_audio.wav,https://storage.googleapis.com/joshtalks-data-...


In [ ]:
df2 = df2.rename(columns={
    'audio': 'audio_file_path',
    'transcription': 'transcription_data'
})

In [ ]:
import json
import os
from pydub import AudioSegment
import re
import pandas as pd
import requests

def is_valid_hindi_text(text):
    """
    Check if text contains only Hindi characters and punctuation marks.
    Hindi Unicode range: \u0900-\u097F (Devanagari)
    Also allows common punctuation and whitespace
    """
    # Define allowed characters: Hindi (Devanagari), whitespace, and common punctuation
    pattern = r'^[\u0900-\u097F\s।,.!?;:()\-]+$'
    return bool(re.match(pattern, text))

def split_segment_if_needed(segment, max_duration=30):
    """
    Split a segment into multiple parts if it exceeds max_duration.
    Returns list of segments with adjusted start/end times.
    """
    duration = segment['end'] - segment['start']

    if duration <= max_duration:
        return [segment]

    # Split into multiple segments
    num_parts = int(duration / max_duration) + 1
    part_duration = duration / num_parts

    segments = []
    for i in range(num_parts):
        new_segment = segment.copy()
        new_segment['start'] = segment['start'] + (i * part_duration)
        new_segment['end'] = segment['start'] + ((i + 1) * part_duration)
        segments.append(new_segment)

    return segments

def clip_audio_from_transcription(audio_file_path, transcription_data, output_base_dir="output"):
    """
    Clip audio file based on transcription data.

    Args:
        audio_file_path: Path to the input audio file (local path)
        transcription_data: List of dictionaries with start, end, speaker_id, and text
        output_base_dir: Base directory in Google Drive for output files

    Returns:
        Tuple of (stats_dict, segments_list)
        - stats_dict: Dictionary with statistics about the processing
        - segments_list: List of dictionaries with segment info (clip_path, text)
    """
    # Get audio filename without extension for folder naming
    audio_filename = os.path.splitext(os.path.basename(audio_file_path))[0]

    # Create output directories specific to this audio file within the Google Drive base directory
    audio_output_dir = os.path.join("/content/drive/MyDrive/", output_base_dir, audio_filename)
    valid_dir = os.path.join(audio_output_dir, "valid_clips")
    invalid_dir = os.path.join(audio_output_dir, "invalid_clips")
    os.makedirs(valid_dir, exist_ok=True)
    os.makedirs(invalid_dir, exist_ok=True)

    # Load the audio file
    print(f"\nProcessing: {audio_file_path}")
    try:
        audio = AudioSegment.from_file(audio_file_path)
    except Exception as e:
        print(f"Error loading audio file: {e}")
        return {
            "audio_file": audio_filename,
            "status": "error",
            "error": str(e),
            "valid_clips": 0,
            "invalid_clips": 0
        }, []

    # Track clip counts per speaker
    speaker_clip_counts = {}
    valid_count = 0
    invalid_count = 0
    segments_info = []

    # Process each transcription segment
    for segment in transcription_data:
        speaker_id = segment['speaker_id']
        text = segment['text']

        # Split segment if longer than 30 seconds
        split_segments = split_segment_if_needed(segment, max_duration=30)

        for sub_segment in split_segments:
            start_time = sub_segment['start']
            end_time = sub_segment['end']

            # Initialize speaker count if not exists
            if speaker_id not in speaker_clip_counts:
                speaker_clip_counts[speaker_id] = 0

            speaker_clip_counts[speaker_id] += 1
            clip_number = speaker_clip_counts[speaker_id]

            # Create clip filename
            clip_filename = f"{speaker_id}_{clip_number}.wav"

            # Check if text is valid Hindi
            is_valid = is_valid_hindi_text(text)

            # Output directory selection
            if is_valid:
                output_path = os.path.join(valid_dir, clip_filename)
                valid_count += 1
            else:
                output_path = os.path.join(invalid_dir, clip_filename)
                invalid_count += 1
                print(f"  Invalid text in {clip_filename}: {text[:50]}...")

            # Extract audio clip (convert seconds to milliseconds)
            start_ms = int(start_time * 1000)
            end_ms = int(end_time * 1000)

            try:
                audio_clip = audio[start_ms:end_ms]
                audio_clip.export(output_path, format="wav")

                # Store segment information
                segments_info.append({
                    "clip_path": output_path,
                    "transcript": text,
                    "speaker_id": speaker_id,
                    "start_time": start_time,
                    "end_time": end_time,
                    "duration": end_time - start_time,
                    "is_valid_hindi": is_valid,
                    "original_audio": audio_filename
                })

            except Exception as e:
                print(f"  Error creating clip {clip_filename}: {e}")
                continue

    stats = {
        "audio_file": audio_filename,
        "status": "success",
        "valid_clips": valid_count,
        "invalid_clips": invalid_count,
        "total_clips": valid_count + invalid_count,
        "speakers": len(speaker_clip_counts),
        "output_dir": audio_output_dir
    }

    print(f"  ✓ Created {valid_count} valid clips, {invalid_count} invalid clips")
    print(f"  Output: {audio_output_dir}")

    return stats, segments_info

def fetch_transcription_from_url(url):
    """
    Fetch transcription data from a URL (JSON file).
    Args:
        url: URL to the JSON file containing transcription data
    Returns:
        List of transcription segments or None if error
    """
    try:
        response = requests.get(url, timeout=30)
        response.raise_for_status()
        transcription = response.json()
        return transcription
    except requests.exceptions.RequestException as e:
        print(f"  Error fetching transcription from {url}: {e}")
        return None
    except json.JSONDecodeError as e:
        print(f"  Error parsing JSON from {url}: {e}")
        return None

def process_dataframe(df, audio_col="audio_file_path", transcription_col="transcription_data",
                     output_base_dir="output"):
    """
    Process a pandas DataFrame with audio files and transcription data.
    Args:
        df: Pandas DataFrame
        audio_col: Name of column containing audio file paths
        transcription_col: Name of column containing transcription data URLs or data
        output_base_dir: Base directory in Google Drive for all output files

    Returns:
        Tuple of (summary_df, segments_df)
        - summary_df: DataFrame with processing statistics per audio file
        - segments_df: DataFrame with each audio segment and its transcript
    """
    results = []
    all_segments = []

    print(f"Processing {len(df)} audio files...")
    print("=" * 60)

    for idx, row in df.iterrows():
        audio_path = row[audio_col]
        transcription = row[transcription_col]

        # Check if transcription is a URL
        if isinstance(transcription, str) and (transcription.startswith('http://') or
                                               transcription.startswith('https://')):
            print(f"\nFetching transcription from URL...")
            transcription = fetch_transcription_from_url(transcription)
            if transcription is None:
                results.append({
                    "audio_file": os.path.basename(audio_path),
                    "status": "error",
                    "error": "Failed to fetch transcription from URL",
                    "valid_clips": 0,
                    "invalid_clips": 0
                })
                continue

        # Handle transcription data if it's a JSON string (not URL)
        elif isinstance(transcription, str):
            try:
                transcription = json.loads(transcription)
            except json.JSONDecodeError as e:
                print(f"Error parsing transcription for {audio_path}: {e}")
                results.append({
                    "audio_file": os.path.basename(audio_path),
                    "status": "error",
                    "error": "Invalid JSON in transcription",
                    "valid_clips": 0,
                    "invalid_clips": 0
                })
                continue

        # Process the audio file and collect segment info
        stats, segments = clip_audio_from_transcription(audio_path, transcription, output_base_dir)
        results.append(stats)
        all_segments.extend(segments)

    print("\n" + "=" * 60)
    print("Processing Complete!")

    # Create summary DataFrame
    results_df = pd.DataFrame(results)

    # Create segments DataFrame
    segments_df = pd.DataFrame(all_segments)

    # Print summary
    print(f"\nSummary:")
    print(f"  Total audio files processed: {len(results_df)}")
    print(f"  Successful: {(results_df['status'] == 'success').sum()}")
    print(f"  Errors: {(results_df['status'] == 'error').sum()}")
    print(f"  Total valid clips: {results_df['valid_clips'].sum()}")
    print(f"  Total invalid clips: {results_df['invalid_clips'].sum()}")
    print(f"  Total segments: {len(segments_df)}")

    return results_df, segments_df

# Example usage
if __name__ == "__main__":

    data = df2

    # Process all audio files in the DataFrame
    # Specify the Google Drive path for output_base_dir
    results_df,segment_df = process_dataframe(data, output_base_dir="Whisper_Finetune/output_clips")

    # Save results to CSV in Google Drive
    results_df.to_csv("/content/drive/MyDrive/Whisper_Finetune/processing_results.csv", index=False)
    segment_df.to_csv("/content/drive/MyDrive/Whisper_Finetune/segments_results.csv", index=False)

    print(f"\nResults saved to: /content/drive/MyDrive/Whisper_Finetune/processing_results.csv and segments_results.csv")

    # Display results
    print("\nDetailed Results:")
    print(results_df)

/usr/local/lib/python3.12/dist-packages/pydub/utils.py:300: SyntaxWarning: invalid escape sequence '\('
  m = re.match('([su]([0-9]{1,2})p?) \(([0-9]{1,2}) bit\)$', token)
/usr/local/lib/python3.12/dist-packages/pydub/utils.py:301: SyntaxWarning: invalid escape sequence '\('
  m2 = re.match('([su]([0-9]{1,2})p?)( \(default\))?$', token)
/usr/local/lib/python3.12/dist-packages/pydub/utils.py:310: SyntaxWarning: invalid escape sequence '\('
  elif re.match('(flt)p?( \(default\))?$', token):
/usr/local/lib/python3.12/dist-packages/pydub/utils.py:314: SyntaxWarning: invalid escape sequence '\('
  elif re.match('(dbl)p?( \(default\))?$', token):


Processing 104 audio files...

Fetching transcription from URL...

Processing: audio_files/825780_audio.wav
  ✓ Created 24 valid clips, 0 invalid clips
  Output: output_clips/825780_audio

Fetching transcription from URL...

Processing: audio_files/825727_audio.wav
  Invalid text in 291038_33.wav: REDACTED...
  ✓ Created 37 valid clips, 1 invalid clips
  Output: output_clips/825727_audio

Fetching transcription from URL...

Processing: audio_files/988596_audio.wav
  Invalid text in 246004_2.wav: REDACTED...
  ✓ Created 28 valid clips, 1 invalid clips
  Output: output_clips/988596_audio

Fetching transcription from URL...

Processing: audio_files/990175_audio.wav
  ✓ Created 25 valid clips, 0 invalid clips
  Output: output_clips/990175_audio

Fetching transcription from URL...

Processing: audio_files/526266_audio.wav
  Invalid text in 286851_12.wav: REDACTED...
  Invalid text in 286851_34.wav: REDACTED...
  ✓ Created 38 valid clips, 2 invalid clips
  Output: output_clips/526266_audio



## **Data Processing Checkpoint**
### Pre processing part has been completed and audio clips for training have been achieved.
### New Model trainings can be resumed from this point since the audio data has been processed according to model configuration and saved to disk for future use

In [ ]:
import pandas as pd
segment_df = pd.read_csv('/content/drive/MyDrive/Whisper_Finetune/segments_results.csv')

In [ ]:
segment_df.head(10)

,clip_path,transcript,speaker_id,start_time,end_time,duration,is_valid_hindi,original_audio
0,output_clips/825780_audio/valid_clips/245746_1...,अब काफी अच्छा होता है क्योंकि उनकी जनसंख्या बह...,245746,0.11,14.42,14.31,True,825780_audio
1,output_clips/825780_audio/valid_clips/245746_2...,अनुभव करके कुछ लिखना था तो वह तो बिना देखिए नह...,245746,14.42,29.03,14.61,True,825780_audio
2,output_clips/825780_audio/valid_clips/245746_3...,जंगल का सफर होता है जब हम रहने के लिए गए थे ना...,245746,29.03,41.84,12.81,True,825780_audio
3,output_clips/825780_audio/valid_clips/245746_4...,पहली बारी था क्योंकि चलना नहीं आता न वहाँ का ज...,245746,42.47,50.57,8.10,True,825780_audio
4,output_clips/825780_audio/valid_clips/245746_5...,हां तो फिर वहां जो दिन भर तो दिन में तो खोजने ...,245746,52.70,66.83,14.13,True,825780_audio
5,output_clips/825780_audio/valid_clips/245746_6...,अजीब सा आवाज आने लगा बहुत अजीब सा डर तो इतना ल...,245746,66.83,80.51,13.68,True,825780_audio
6,output_clips/825780_audio/valid_clips/245746_7...,डर लगता है लेकिन बहुत सारे थे तो पता सब अपना अ...,245746,89.06,103.19,14.13,True,825780_audio
7,output_clips/825780_audio/valid_clips/245746_8...,ऐसा ही बात था तो पता है रात को मतलब जैसे छः सा...,245746,103.19,117.29,14.10,True,825780_audio
8,output_clips/825780_audio/valid_clips/245746_9...,लगता है कि सर सुबह हो गया लेकिन कोई उठाने वाला...,245746,117.38,129.92,12.54,True,825780_audio
9,output_clips/825780_audio/valid_clips/245746_1...,हम्म हम लोग की तो सुबह होती है की मम्मी उठाने ...,245746,130.25,144.83,14.58,True,825780_audio


In [ ]:
final_df = segment_df[segment_df['is_valid_hindi'] == True]
final_df.head(20)

,clip_path,transcript,speaker_id,start_time,end_time,duration,is_valid_hindi,original_audio
0,/content/drive/MyDrive/Whisper_Finetune/output...,अब काफी अच्छा होता है क्योंकि उनकी जनसंख्या बह...,245746,0.11,14.42,14.31,True,825780_audio
1,/content/drive/MyDrive/Whisper_Finetune/output...,अनुभव करके कुछ लिखना था तो वह तो बिना देखिए नह...,245746,14.42,29.03,14.61,True,825780_audio
2,/content/drive/MyDrive/Whisper_Finetune/output...,जंगल का सफर होता है जब हम रहने के लिए गए थे ना...,245746,29.03,41.84,12.81,True,825780_audio
3,/content/drive/MyDrive/Whisper_Finetune/output...,पहली बारी था क्योंकि चलना नहीं आता न वहाँ का ज...,245746,42.47,50.57,8.10,True,825780_audio
4,/content/drive/MyDrive/Whisper_Finetune/output...,हां तो फिर वहां जो दिन भर तो दिन में तो खोजने ...,245746,52.70,66.83,14.13,True,825780_audio
5,/content/drive/MyDrive/Whisper_Finetune/output...,अजीब सा आवाज आने लगा बहुत अजीब सा डर तो इतना ल...,245746,66.83,80.51,13.68,True,825780_audio
6,/content/drive/MyDrive/Whisper_Finetune/output...,डर लगता है लेकिन बहुत सारे थे तो पता सब अपना अ...,245746,89.06,103.19,14.13,True,825780_audio
7,/content/drive/MyDrive/Whisper_Finetune/output...,ऐसा ही बात था तो पता है रात को मतलब जैसे छः सा...,245746,103.19,117.29,14.10,True,825780_audio
8,/content/drive/MyDrive/Whisper_Finetune/output...,लगता है कि सर सुबह हो गया लेकिन कोई उठाने वाला...,245746,117.38,129.92,12.54,True,825780_audio
9,/content/drive/MyDrive/Whisper_Finetune/output...,हम्म हम लोग की तो सुबह होती है की मम्मी उठाने ...,245746,130.25,144.83,14.58,True,825780_audio


In [ ]:
# Logging in to hugging face to upload model
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
data_df = final_df[['clip_path', 'transcript']]
data_df = data_df.rename(columns={
    'clip_path': 'audio',
    'transcript': 'sentence'
})
data_df.head(5)

,audio,sentence
0,/content/drive/MyDrive/Whisper_Finetune/output...,अब काफी अच्छा होता है क्योंकि उनकी जनसंख्या बह...
1,/content/drive/MyDrive/Whisper_Finetune/output...,अनुभव करके कुछ लिखना था तो वह तो बिना देखिए नह...
2,/content/drive/MyDrive/Whisper_Finetune/output...,जंगल का सफर होता है जब हम रहने के लिए गए थे ना...
3,/content/drive/MyDrive/Whisper_Finetune/output...,पहली बारी था क्योंकि चलना नहीं आता न वहाँ का ज...
4,/content/drive/MyDrive/Whisper_Finetune/output...,हां तो फिर वहां जो दिन भर तो दिन में तो खोजने ...


In [ ]:
!uv pip install --quiet evaluate tensorboard librosa gradio

In [ ]:
!uv pip install --quiet jiwer

In [ ]:
# !pip install --upgrade datasets[audio] transformers accelerate evaluate jiwer tensorboard gradio librosa

In [ ]:
import pandas as pd
from datasets import Dataset, DatasetDict
import torch
from transformers import (
    WhisperForConditionalGeneration,
    WhisperProcessor,
    Seq2SeqTrainingArguments,
    Seq2SeqTrainer,
)
import evaluate
from dataclasses import dataclass
from typing import Any, Dict, List, Union
import librosa

# 1. Clearing CUDA cache before model training
if torch.cuda.is_available():
    torch.cuda.empty_cache()

# 2. Mounting Google Drive for loading data file
gdrive_path = "/content/drive/MyDrive/Whisper_Finetune/"

# 3. Create Hugging Face Dataset from the pandas DataFrame
dataset = Dataset.from_pandas(data_df)

# Removing the pandas DataFrame to free up memory
del data_df

# Splitting the dataset into train and test datasets
train_test_split = dataset.train_test_split(test_size=0.1)
common_voice_datasets = DatasetDict({
    "train": train_test_split["train"],
    "test": train_test_split["test"],
})

# 4. Loading Processor
model_name = "openai/whisper-small"
processor = WhisperProcessor.from_pretrained(model_name, language="hindi", task="transcribe")

# 5. Pre-process Data (with on-the-fly audio loading)
def prepare_dataset(batch):
    # Load and resample audio data from the file path
    audio = batch["audio"]
    try:
        # librosa.load returns a numpy array and the sampling rate
        audio_array, sampling_rate = librosa.load(audio, sr=16000)
        batch["input_features"] = processor(audio_array, sampling_rate=sampling_rate).input_features[0]
    except Exception as e:
        print(f"Error processing {audio}: {e}")
        # In case of an error, you might want to return an empty feature set or handle it appropriately
        batch["input_features"] = []


    # Process the transcriptions with proper padding token handling
    batch["labels"] = processor.tokenizer(
        batch["sentence"],
        truncation=True,
        max_length=448  # Match Whisper's max length
    ).input_ids

    return batch

# Use .map() to apply the function. Audio is loaded and processed here.
# The original 'audio' and 'sentence' columns are kept for now.
# You can set remove_columns later if needed.
# NEW CODE FOR DEBUGGING
print("Mapping dataset and processing audio on-the-fly (with a single process for debugging)...")
common_voice_datasets = common_voice_datasets.map(prepare_dataset) # num_proc removed, defaults to 1
print("Dataset mapping complete.")


# 6. Data Collator and Metrics (same as before)
@dataclass
class DataCollatorSpeechSeq2SeqWithPadding:
    processor: Any

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # Split inputs and labels since they need different padding
        input_features = [{"input_features": feature["input_features"]} for feature in features]
        label_features = [{"input_ids": feature["labels"]} for feature in features]
        # Pad input features
        batch = self.processor.feature_extractor.pad(input_features, return_tensors="pt")
        # Pad labels using tokenizer in one call (this is the efficient way)
        labels_batch = self.processor.tokenizer.pad(label_features, return_tensors="pt")
        # Replace padding with -100 to ignore in loss
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)
        # Remove decoder_input_ids if present
        if (labels[:, 0] == self.processor.tokenizer.bos_token_id).all().cpu().item():
            labels = labels[:, 1:]

        batch["labels"] = labels
        return batch

data_collator = DataCollatorSpeechSeq2SeqWithPadding(processor=processor)
metric = evaluate.load("wer")

def compute_metrics(pred):
    pred_ids = pred.predictions
    label_ids = pred.label_ids
    label_ids[label_ids == -100] = processor.tokenizer.pad_token_id
    pred_str = processor.tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    label_str = processor.tokenizer.batch_decode(label_ids, skip_special_tokens=True)
    wer = 100 * metric.compute(predictions=pred_str, references=label_str)
    return {"wer": wer}

# 7. Loading Model
model = WhisperForConditionalGeneration.from_pretrained(model_name)
# You can remove 'model.config.use_cache = False' as it's not needed for this solution.
model.generation_config.language = "hindi"
model.generation_config.task = "transcribe"
model.generation_config.forced_decoder_ids = None
# 8. Define Training Arguments - ALTERNATE SOLUTION
training_args = Seq2SeqTrainingArguments(
    output_dir=gdrive_path + "whisper-small-hi-custom",
    # 1. Reducing batch size to prevent out-of-memory errors.
    per_device_train_batch_size=8,
    # 2. Disabling gradient checkpointing, which was caused error.
    gradient_checkpointing=False,
    # 3. Using gradient accumulation to maintain an effective batch size of 16 (8 * 2).
    gradient_accumulation_steps=2,
    fp16=True,
    learning_rate=1e-5,
    warmup_steps=500,
    # 4. Changing from 4000 -> 2000 for avoiding overfiting and faster training time
    max_steps=2000,
    eval_strategy="steps",
    per_device_eval_batch_size=8,
    predict_with_generate=True,
    generation_max_length=225,
    save_steps=500,
    eval_steps=500,
    logging_steps=25,
    report_to=["tensorboard"],
    load_best_model_at_end=True,
    metric_for_best_model="wer",
    greater_is_better=False,
    push_to_hub=False,
)

# 9. Initializing Trainer
trainer = Seq2SeqTrainer(
    args=training_args,
    model=model,
    train_dataset=common_voice_datasets["train"],
    eval_dataset=common_voice_datasets["test"],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=processor.feature_extractor,
)

# 10. Starting Training
print("\nStarting fine-tuning on Colab GPU...")
trainer.train()
print("Fine-tuning completed.")

# 11. Saving the final model to Google Drive
trainer.save_model(gdrive_path + "whisper-small-hi-custom-final")
print("Model saved to Google Drive.")

preprocessor_config.json: 0.00B [00:00, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

...


Starting fine-tuning on Colab GPU...


You're using a WhisperTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss,Validation Loss,Wer
500,0.335600,0.373359,40.376437
1000,0.142700,0.334092,36.023889
1500,0.097700,0.358715,36.241064
2000,0.051300,0.381412,35.870057


The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
/usr/local/lib/python3.12/dist-packages/transformers/modeling_utils.py:3918: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 448, 'suppress_tokens': [1, 2, 7, 8, 9, 10, 14, 25, 26, 27, 28, 29, 31, 58, 59, 60, 61, 62, 63, 90, 91, 92, 93, 359, 503, 522, 542, 873, 893, 902, 918, 922, 931, 1350, 1853, 1982, 2460, 2627, 3246, 3253, 3268, 3536, 3846, 3961, 4183, 4667, 6585, 6647, 7273, 9061, 9383, 10428, 10929, 11938, 12033, 12331, 12562, 13793, 14157, 14635, 15265, 15618, 16553, 16604, 18362, 18956, 20075, 21675, 22520, 26130, 26161, 26435, 28279, 29464, 31650, 32302, 32470, 36865, 42863, 47425, 49870, 50254, 50258, 50360, 50361, 50362], 'begin_suppress_tokens': [220, 50257]}. You are seeing this warning because yo

Fine-tuning completed.
Model saved to Google Drive.


In [ ]:
# After training, saving the processor and tokenizer
gdrive_path = "/content/drive/MyDrive/Whisper_Finetune_NEW/"
trainer.save_model(gdrive_path + "whisper-small-hi-custom-final-new")
processor.save_pretrained(gdrive_path + "whisper-small-hi-custom-final-new")
print("Model and processor saved to Google Drive.")


Model and processor saved to Google Drive.


In [ ]:
trainer.push_to_hub("Pranav13/whisper-small-hi-custom-final-new")
processor.push_to_hub("Pranav13/whisper-small-hi-custom-final-new")

Processing Files (0 / 0)      : |          |  0.00B /  0.00B            

New Data Upload               : |          |  0.00B /  0.00B            

  ...1615.0576d851f563.12113.0: 100%|##########| 6.84kB / 6.84kB            

  ...246284.140382b3ea58.240.0: 100%|##########| 20.8kB / 20.8kB            

  ...005569.fd3ba1b3c46e.605.0: 100%|##########| 28.8kB / 28.8kB            

...

  ...3107.66139171915f.12794.0: 100%|##########| 6.84kB / 6.84kB            

  ...-custom/model.safetensors:   0%|          |  570kB /  967MB            

  ...414764.9a69a66ca4ed.374.0:  16%|#6        | 4.12kB / 25.3kB            

  ...-custom/training_args.bin:  16%|#6        |   992B / 6.10kB            

CommitInfo(commit_url='https://huggingface.co/Pranav13/whisper-small-hi-custom-final-new/commit/972d496395e9ebd559072237deeabad13e59dc83', commit_message='Upload processor', commit_description='', oid='972d496395e9ebd559072237deeabad13e59dc83', pr_url=None, repo_url=RepoUrl('https://huggingface.co/Pranav13/whisper-small-hi-custom-final-new', endpoint='https://huggingface.co', repo_type='model', repo_id='Pranav13/whisper-small-hi-custom-final-new'), pr_revision=None, pr_num=None)

In [ ]:
model.push_to_hub("Pranav13/whisper-small-hi-custom-final-new")
processor.push_to_hub("Pranav13/whisper-small-hi-custom-final-new")

Processing Files (0 / 0)      : |          |  0.00B /  0.00B            

New Data Upload               : |          |  0.00B /  0.00B            

  ...9fyapee/model.safetensors:   3%|3         | 33.5MB /  967MB            

No files have been modified since last commit. Skipping to prevent empty commit.
No files have been modified since last commit. Skipping to prevent empty commit.


CommitInfo(commit_url='https://huggingface.co/Pranav13/whisper-small-hi-custom-final-new/commit/837e938d663f586c28fc95f9c3aa98cf4eb5eca0', commit_message='Upload processor', commit_description='', oid='837e938d663f586c28fc95f9c3aa98cf4eb5eca0', pr_url=None, repo_url=RepoUrl('https://huggingface.co/Pranav13/whisper-small-hi-custom-final-new', endpoint='https://huggingface.co', repo_type='model', repo_id='Pranav13/whisper-small-hi-custom-final-new'), pr_revision=None, pr_num=None)